In [1]:
import os
import pandas as pd
import numpy as np
from pathlib import Path

if not "root" in locals():
    current_path = Path(os.getcwd())
    root = current_path.parent.absolute()

In [2]:
os.chdir(root)
from data_treatment.box_creator import create_binary_box

df = pd.read_csv('database.csv')
df = create_binary_box(df, threshold=2., box_length=7).set_index("Date").dropna(axis = 0)
df

,PX_OPEN_VIX_volatility,PX_HIGH_VIX_volatility,PX_LOW_VIX_volatility,PX_LAST_VIX_volatility,VOLUME_TOTAL_CALL_VIX_volatility,PUT_CALL_VOLUME_RATIO_CUR_DAY_VIX_volatility,VOLUME_TOTAL_PUT_VIX_volatility,TOT_OPT_VOLUME_CUR_DAY_VIX_volatility,OPEN_INT_TOTAL_CALL_VIX_volatility,OPEN_INT_TOTAL_PUT_VIX_volatility,...,S&P 500 Weekly Close_SENTIMENT,ST_Rev_F-F_ST_Reversal_Factor_daily,Mkt-RF_F-F_Research_Data_5_Factors_2x3_daily,SMB_F-F_Research_Data_5_Factors_2x3_daily,HML_F-F_Research_Data_5_Factors_2x3_daily,RMW_F-F_Research_Data_5_Factors_2x3_daily,CMA_F-F_Research_Data_5_Factors_2x3_daily,RF_F-F_Research_Data_5_Factors_2x3_daily,Mom _F-F_Momentum_Factor_daily,Box
Date,,,,,,,,,,,,,,,,,,,,,
2006-02-28,11.74,12.36,11.71,12.34,3086.0,1.7498,5400.0,8486.0,11155.0,4736.0,...,1292.67,-0.01,-1.10,-0.18,0.10,0.07,0.13,0.018,-0.39,0
2006-03-01,12.05,12.07,11.52,11.54,1879.0,0.6663,1252.0,3131.0,14207.0,8921.0,...,1292.67,0.16,0.90,0.61,-0.02,-0.34,0.36,0.016,0.85,0
2006-03-02,11.79,11.95,11.56,11.72,6747.0,0.3587,2420.0,9167.0,15874.0,9018.0,...,1291.24,0.41,-0.16,-0.04,0.09,0.00,0.04,0.016,0.45,0
2006-03-03,12.03,12.10,11.36,11.96,1099.0,21.0373,23120.0,24219.0,30647.0,11618.0,...,1291.24,-0.17,-0.15,-0.01,-0.04,0.13,0.01,0.016,0.01,0
2006-03-06,12.22,12.91,12.22,12.74,4217.0,0.1859,784.0,5001.0,35130.0,34015.0,...,1291.24,-0.56,-0.75,-0.28,-0.11,-0.19,0.01,0.016,-0.95,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-18,18.89,21.45,18.81,21.34,435445.0,0.8205,357293.0,792738.0,6190495.0,4299845.0,...,4063.04,1.13,-0.69,0.06,-0.93,-1.01,-0.42,0.000,-0.26,1
2021-05-19,22.46,25.96,21.88,22.18,417537.0,1.2020,501893.0,919430.0,4725251.0,3133065.0,...,4063.04,1.02,-0.34,-0.49,-0.46,-0.18,-0.34,0.000,-0.70,0
2021-05-20,22.33,23.50,20.19,20.67,208977.0,0.8944,186913.0,395890.0,4842985.0,3207980.0,...,4137.60,1.24,1.15,-0.47,-1.43,-0.84,-0.49,0.000,0.20,0


In [3]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.feature_selection import SelectFromModel

# prescale_X = df.drop(columns="Box")
# y = df["Box"]
# X = sm.add_constant((prescale_X - prescale_X.mean())/prescale_X.std())


# lsvc = LogisticRegression(penalty="l1", solver="liblinear", C = 0.01).fit(X, y)
# model = SelectFromModel(lsvc, prefit=True)
# X_new = model.transform(X)
# X_new.shape

In [4]:
# X.columns[model.get_support()]

Analyse des features pertinentes en fonction du temps

In [5]:
window = 245 #Selection des meilleures colonnes par fenêtres de 1 an glissantes
step = 1
df.reset_index(level=0, inplace=True)


In [8]:
for i in range(df.shape[0] - window):
    sub_df = df.iloc[i : i + window]
    if i == 600:
        print(sub_df)
        break

           Date  PX_OPEN_VIX_volatility  PX_HIGH_VIX_volatility  \
600  2010-04-23                   16.62                   16.71   
601  2010-07-22                   24.37                   24.85   
602  2010-07-23                   24.79                   25.17   
603  2010-10-21                   19.70                   20.53   
604  2010-10-22                   19.36                   19.36   
..          ...                     ...                     ...   
840  2014-11-28                   12.64                   13.49   
841  2014-12-01                   14.16                   14.75   
842  2014-12-02                   14.10                   14.17   
843  2014-12-03                   12.75                   12.88   
844  2014-12-04                   12.70                   13.23   

     PX_LOW_VIX_volatility  PX_LAST_VIX_volatility  \
600                  16.12                   16.62   
601                  23.72                   24.63   
602                  23.32       